# python과 html을 분리하기

# 1. 템플릿 개념
- @out
- html을 제일 위로, 파이선 코드를 아래로 옯기고
- replace 함수 이용

In [1]:
# test2.py (templit.py)
'''
html = """
<html>
   <head>
      <meta charset="utf-8">
   </head>

   <body>
       <font color=red> @out</font>
   </body>
</html>
"""

html = html.replace("@out",  "제목출력")

print(html)
'''


'\nhtml = """\n<html>\n   <head>\n      <meta charset="utf-8">\n   </head>\n\n   <body>\n       <font color=red> @out</font>\n   </body>\n</html>\n"""\n\nhtml = html.replace("@out",  "제목출력")\n\nprint(html)\n'

In [ ]:
import socket
import threading
from datetime import datetime
import subprocess

def httpprocess(client_socket) :
    data = client_socket.recv(65535)   
    data = data.decode()
    print(data)
    try :    
        headers = data.split("\r\n")
        filename = headers[0].split(" ")[1]
        
        if '.py' in filename:
            
            html = subprocess.check_output(['python.exe', '.' + filename])
            html = html.decode('cp949')
            
            header = 'HTTP/1.0 200 OK\r\n'               
            client_socket.send(header.encode("utf-8"))
            client_socket.send("Content-Type: text/html\r\n\r\n".encode())                                    
            client_socket.send(html.encode("utf-8"))            
        elif '.html' in filename:
            file = open("."+ filename, 'rt', encoding='utf-8')
            html = file.read()    
            header = 'HTTP/1.0 200 OK\r\n'               
            client_socket.send(header.encode("utf-8"))
            client_socket.send("Content-Type: text/html\r\n\r\n".encode())            
            client_socket.send(html.encode("utf-8"))
        elif '.jpg' in filename or '.png' in filename:         
            client_socket.send('HTTP/1.1 200 OK\r\n'.encode())
            client_socket.send("Content-Type: image/jpg\r\n".encode())
            client_socket.send("Accept-Ranges: bytes\r\n\r\n".encode())
            file = open("." + filename, "rb")            
            client_socket.send(file.read())  
            file.close()               
        else :
            header = 'HTTP/1.0 404 File Not Found\r\n\r\n'        
            client_socket.send(header.encode("utf-8"))
    except Exception as e :
        print(e)         
    client_socket.close()


server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 80))
server_socket.listen(0)
print("listening")

while  True :
    client_socket, addr = server_socket.accept()
    print("accepting")
    t = threading.Thread(target=httpprocess, args=(client_socket,))
    t.start()

listening
accepting
GET /5.engine.py HTTP/1.1
Host: 127.0.0.1
Connection: keep-alive
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36
Sec-Fetch-User: ?1
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9
Sec-Fetch-Site: none
Sec-Fetch-Mode: navigate
Accept-Encoding: gzip, deflate, br
Accept-Language: ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7

accepting


list index out of range


> 브라우저에서 
- http://127.0.0.1/templit.py

In [ ]:
# templit2.py
'''
html = """
<html>
   <head>
      <meta charset="utf-8">
   </head>

   <body>
       <font color=red> @out</font>
       <table borker = 2>
            <tr>
                <td> 이름</td> <td> @name </td>
            </tr>
            <tr>
                <td> e-mail</td> <td> @email </td>
            </tr>
              
   </body>
</html>
"""

html = html.replace("@out",  "제목출력")
html = html.replace("@name",  "이순신")
html = html.replace("@email",  "lee@gmail.com")


print(html)
'''

#### 템플릿 개념


In [1]:
html = 'hello@v1  test@v2   item@v3'

html = html.replace('@v1', '안녕하세요')
html = html.replace('@v2', '이순신님')
html = html.replace('@v3', '거북선')
html

'hello안녕하세요  test이순신님   item거북선'

# 2. 엔진 개념
- dict 이용

- py파일과 html파일을 따로 분리해서 관리

In [26]:
def render(html, data):
    for v in data:
        html = html.replace('@'+v, data[v])
    return html

def renderfile(file, data):
    html = open(file, 'rt', encoding = 'utf=8').read()
    for v in data:
        html = html.replace('@' + v, data[v])
    return html
        


In [27]:
html = 'hello@v1  test@v2   item@v3'

data = {'v1': '안녕하세요',
       'v2': '이순신',
       'v3': '거북선'}

html = render(html, data)
html

'hello안녕하세요  test이순신   item거북선'

In [30]:
data = {"title":"나의 홈페이지",  "name":"이순신", "email":"^^"}


print(renderfile('template.html', data))


<!-- saved from url=(0031)http://10.5.4.100/template.html -->
<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
      
   </head>

   <body>
       <font color="blue"> 나의 홈페이지</font>
     <table border="1">
             <tbody><tr bgcolor="yellow">
                  <td> 이름 </td>  <td> 이순신 </td> 
             </tr>  
             <tr>
                  <td> email </td>  <td> ^^ </td> 
             </tr>
     </tbody></table>
   
</body></html>


In [ ]:
# template.html
'''
<!-- saved from url=(0031)http://10.5.4.100/template.html -->
<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
      
   </head>

   <body>
       <font color="blue"> @title</font>
     <table border="1">
             <tbody><tr bgcolor="yellow">
                  <td> 이름 </td>  <td> @name </td> 
             </tr>  
             <tr>
                  <td> email </td>  <td> @email </td> 
             </tr>
     </tbody></table>
   
</body></html>
'''

In [ ]:
# engine.py
'''
def renderfile(file, data) :
    html = open(file, "rt", encoding="utf-8").read()
    for v in data :
        html = html.replace("@"+v, data[v])
    return html

data = {"title":"나의 홈페이지",  "name":"이순신", "email":"lee@gmail.com"}
print(renderfile("template.html", data))
''''''

> 브라우저에서
- http://127.0.0.1/5.engine.py